In [8]:
import os

def load_env_file(filepath=".env"):
    """
    Reads a .env file and sets environment variables.
    """
    try:
        with open(filepath, "r") as f:
            for line in f:
                # Remove whitespace and comments
                line = line.strip()
                if not line or line.startswith('#'):
                    continue

                # Parse key-value pairs
                key, value = line.split('=', 1)
                key = key.strip()
                value = value.strip().strip('"\'') # Handle single/double quotes

                os.environ[key] = value
    except FileNotFoundError:
        print(f"Warning: .env file not found at {filepath}. Skipping.")

# Run the function to load your variables
load_env_file()

# Access the environment variable
api_key = os.getenv('GOOGLE_API_KEY')
# print(f"My API key is: {api_key}")
# print(f"API Key from environment: {os.getenv('GOOGLE_API_KEY')}")

In [9]:
import os
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.gemini import GeminiEmbedding
from llama_index.core import Settings
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent.workflow import ReActAgent
from llama_index.core.workflow import Context
from llama_index.core.agent.workflow import ToolCallResult, AgentStream
from llama_index.core.node_parser import SentenceSplitter
from llama_index.retrievers.bm25 import BM25Retriever

# Or set it directly in the Settings
Settings.llm = Gemini(model="gemini-2.5-pro")
Settings.embed_model = GeminiEmbedding()
Settings.chunk_size = 512
Settings.node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=50)
Settings.embed_model = GeminiEmbedding(model_name="models/embedding-001")


/var/folders/5_/86x7msms3b9d5qzksrmtsl6c0000gn/T/ipykernel_94843/1934742543.py:14: DeprecationWarning: Call to deprecated class Gemini. (Should use `llama-index-llms-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/llm/google_genai/)
  Settings.llm = Gemini(model="gemini-2.5-pro")
/var/folders/5_/86x7msms3b9d5qzksrmtsl6c0000gn/T/ipykernel_94843/1934742543.py:15: DeprecationWarning: Call to deprecated class GeminiEmbedding. (Should use `llama-index-embeddings-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/embeddings/google_genai/)
  Settings.embed_model = GeminiEmbedding()
/var/folders/5_/86x7msms3b9d5qzksrmtsl6c0000gn/T/ipykernel_94843/1934742543.py:18: DeprecationWarning: Call to deprecated class GeminiEmbedding. (Should use `llama-index-embeddings-google-genai` instead, using Google's latest unified SDK. See: https://docs.llamaindex.ai/en/stable/examples/embedding

In [10]:
DIR="ocr"
DIR="test"
SOURCE_DIR=f"./{DIR}"
PERSIST_DIR=f"./storage/{DIR}"

if not os.path.exists(PERSIST_DIR):
    print("Index not found. Building a new index...")
    # Load documents from a directory
    documents = SimpleDirectoryReader(SOURCE_DIR).load_data()
    # Create the VectorStoreIndex
    index = VectorStoreIndex.from_documents(documents)
    # Persist the index to disk
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    print("Loading existing index from storage...")
    # Rebuild storage context
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    # Load the index from the storage context
    index = load_index_from_storage(storage_context)

index_engine = index.as_query_engine(similarity_top_k=5)

# bm25_retriever = BM25Retriever.from_defaults(
#     nodes=list(index.storage_context.docstore.docs.values()),
#     similarity_top_k=5, # Retrieve top 5 keyword results
# )

query_engine_tools = [
    QueryEngineTool.from_defaults(
        query_engine=index_engine,
        name="cyprus_file",
        description=(
            "Report of the Parliamentary Committee on the Cyprus File"
            "Use a detailed plain text question as input to the tool."
        ),
    ),
    # QueryEngineTool.from_defaults(
    #     query_engine=index_engine,
    #     name="cyprus_file",
    #     description=(
    #         "Report of the Parliamentary Committee on the Cyprus File"
    #         "Use a detailed plain text question as input to the tool."
    #     ),
    # ),
]

Index not found. Building a new index...


In [7]:
# --- Chatting ---
# Create a chat engine
# A chat engine is designed for conversational, back-and-forth interactions.
print("⚙️  Creating chat engine...")
chat_engine = index.as_chat_engine(
    chat_mode="context",
    verbose=True,
    system_prompt="You are a helpful assistant that can answer questions about the Cyprus File. " \
    "You can use the provided tools to answer questions. " \
    "If you don't know the answer, you should say 'I don't know'. " \
    "You should always use the tools to answer questions, even if you think you know the answer." \
    "You should always provide a detailed answer, even if the question is simple." \
    "You should always provide a list of key points in your answer." \
    "Always cite the source of your information from the context.\n"
)
print("✅ Chat engine ready. Type 'exit' or 'quit' to end the chat.")
print("-" * 50)

# Start the conversation loop
while True:
    query = input("Ask a question: ")
    if query.lower() in ["quit", "exit"]:
        break
    
    # Get the response from the chat engine
    response = chat_engine.chat(query)
    
    # Print the response
    print("\nGemini:", response)
    print("-" * 50)

⚙️  Creating chat engine...
✅ Chat engine ready. Type 'exit' or 'quit' to end the chat.
--------------------------------------------------


Ask a question:  Who is makarios?



Gemini: Based on the provided information, Makarios is mentioned in the context of the Kofinou incident. The document states that according to witness testimonies, Makarios was not responsible for planning or causing this event.

Here are the key points from the provided text:

*   **Not Responsible for Kofinou Incident:** The document explicitly states that witness testimonies show the Kofinou incident was planned and caused by the Athens junta and "in no case by Makarios".
*   **Context:** This information is presented in a section discussing the Kofinou incident.

**Source:**
*   kipros Porisma Fakelou.pdf, page 48
--------------------------------------------------


Ask a question:  summarise the document



Gemini: Based on the provided context from the document "kipros Porisma Fakelou.pdf," the text appears to be a report or finding concerning various political and historical events in Cyprus. The snippets touch upon different individuals, organizations, and incidents.

Here is a summary of the key points from the provided information:

*   **Political Organizations and Intrigue:** The document discusses an organization named "Hellenic Resistance" (Ελληνική Αντίστασις). It is mentioned in connection with an assassination attempt and is described as being directly linked to an organization in Rome.
    *   **Source:** kipros Porisma Fakelou.pdf, page 54
*   **Key Figures Implicated:** The leaders of the Rome-based organization are identified as Andreas Papandreou and Nikolaidis. The Cypriot minister Yiorkatzis is also noted as being involved with this group.
    *   **Source:** kipros Porisma Fakelou.pdf, page 54
*   **Succession of Grivas:** There is a mention of Georgios Grivas and the

Ask a question:  Diffuculty of acquiring testinony



Gemini: Empty Response
--------------------------------------------------


Ask a question:  can you tell me about the difficulty of acquiring testemony



Gemini: Based on the provided context, the document highlights several instances that point to the difficulty of acquiring complete and confirmed testimony.

Here are the key points regarding the challenges in obtaining testimony:

*   **Inability to Secure Testimony:** The committee explicitly "did not manage to secure the testimony" of Stylianos Pattakos, who was the Minister to the Prime Minister's Office in the junta government. This is a direct example of a key individual's testimony being unobtainable.
    *   **Source:** kipros Porisma Fakelou.pdf, page 48

*   **Unconfirmed Information:** In another instance, the document mentions a report about a small council being
--------------------------------------------------


KeyboardInterrupt: Interrupted by user

In [44]:
agent = ReActAgent(
    tools=query_engine_tools,
    llm=Settings.llm,
    # verbose=True,
    system_prompt="You are a helpful assistant that can answer questions about the Cyprus File. " \
    "You can use the provided tools to answer questions. " \
    "If you don't know the answer, you should say 'I don't know'. " \
    "You should always use the tools to answer questions, even if you think you know the answer." \
    "You should always provide a detailed answer, even if the question is simple." \
    "You should always provide a list of key points in your answer." \
    "Always cite the source of your information from the context.\n"
)

# context to hold this session/state

ctx = Context(agent)

In [45]:
while True:
    user_input = input("Enter your question: ")
    if user_input.lower() in [None, '', "exit", "quit"]:
        print("Exiting the agent.")
        break
    handler = agent.run(user_input, ctx=ctx)

    # async for ev in handler.stream_events():
    #     # if isinstance(ev, ToolCallResult):
    #     #     print_wrapped_response(f"\nCall {ev.tool_name} with {ev.tool_kwargs}\nReturned: {ev.tool_output}")
    #     if isinstance(ev, AgentStream):
    #         print(f"{ev.delta}", end="", flush=True)

    response = await handler
    print(str(response))

The commission, presided over by Christos Basagiannis, held sessions on dates including October 23, 1986, and December 3, 1986. During these sessions, the commission investigated various matters, including the general mobilization in Greece on July 20th and the circumstances surrounding the discharge and subsequent dismissal of a military witness. The broader context of the commission's work involved the coup against Makarios and the role of Greek military forces in Cyprus, which were considered a defensive necessity by the Cypriot side.
Exiting the agent.
